In [1]:
# built-in imports
import re
import pandas as pd
import json
import numpy as np

In [2]:
# read in the csv file
path = "../data/curated/cleaned_property_with_features.csv"
property_data = pd.read_csv(path)

In [3]:
property_data.shape

(14819, 13)

In [4]:
# merge SA2 code and postcode
# df_postcode = pd.read_csv('australian_postcodes.csv')
# vic_postcode = df_postcode[df_postcode['state'] == 'VIC']
# vic_postcode = vic_postcode.loc[:, ['postcode', 'SA2_MAINCODE_2016', 'SA2_NAME_2016']].reset_index(drop=True)
# result = pd.merge(unique_property_df, vic_postcode, how='outer', sort=True, on='postcode')
# result.head()

In [5]:
# get the rent price data
rent_df_new = pd.DataFrame()
rent_df_new["rent"] = property_data["rent"]

# clean the rent data to gain the weekly rent
for i in range(len(rent_df_new["rent"])):

    # get the price
    price = rent_df_new["rent"][i].lower()
    rent = re.findall(r"[$](\d+\.?,?\d*)", price)

    # find the annually rent
    if ("p.a." in price) or ("annually" in price) or ("pa." in price):
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))/52

    # find the fortnightly rent
    elif "fortnightly" in price:
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))/2

    # find the nightly rent
    elif "night" in price:
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)
        rent_df_new["rent"][i] = float(rent[0].replace(",", "")) * 7

    # find the monthly rent
    elif ("pcm" in price) or ("pm" in price) or ("month" in price):

        # check whether contain weekly rent
        if re.findall(r"(p([a-z]*|\/*)\s?w)", price) != "":
            if len(rent) == 0:
                rent = re.findall(r"(\d+\.?,?\d*)", price)
            rent_df_new["rent"][i] = float(rent[0].replace(",", ""))
            
        # if only contain monthly rent, calculate the weekly rent
        else:
            if len(rent) == 0:
                rent = re.findall(r"(\d+\.?,?\d*)", price)
            rent = (float(rent[0].replace(",", "")) / 30) * 7
            rent_df_new["rent"][i] = rent
        
    # find the weekly rent
    else:
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)

            # set the rent without any price data as zero
            if len(rent) == 0:
                rent_df_new["rent"][i] = 0
            else:
                rent_df_new["rent"][i] = float(rent[0].replace(",", ""))

        elif len(rent) == 2:
            num1 = int(float(rent[0].replace(",", "")))
            num2 = int(float(rent[1].replace(",", "")))

            # check for normal weekly rent range
            if len(str(num1)) != len(str(num2)):
                # check whether the larger price is twice than the lower price, 
                # if yes, using lower price, if no, using the mean price
                if num1 * 2 < num2:
                    rent = num1
                else:
                    rent = (num1 + num2)/2
            else:
                rent = (num1 + num2)/2
            rent_df_new["rent"][i] = rent

        else:
            rent_df_new["rent"][i] = float(rent[0].replace(",", ""))


# save two decimal places
# formater="{0:.02f}".format
# rent_df_new = rent_df_new.applymap(formater)

# transfer the rent data type to integer
rent_df_new = rent_df_new.astype(int)

In [6]:
rent_df_new.shape

(14819, 1)

In [7]:
# add the processed rent data into the whole data set
property_data["rent_weekly"] = rent_df_new["rent"]

In [8]:
property_data.shape

(14819, 14)

In [12]:
# save the rent cleaned data
property_data.to_csv("../data/curated/weekly_rent.csv", index = None)

In [10]:
drop_zero_rent = property_data[property_data["rent_weekly"] != 0]

In [11]:
drop_zero_rent.shape

(14458, 14)

In [13]:
drop_zero_rent.to_csv("../data/curated/cleaned_rent.csv", index = None)